In [1]:
!pip install sortedcontainers
!pip install git+https://github.com/ihler/pyGMs.git
!pip install pyro-ppl


  Cloning https://github.com/ihler/pyGMs.git to c:\users\micha\appdata\local\temp\pip-req-build-sek7g4cz
  Resolved https://github.com/ihler/pyGMs.git to commit 600d48493b868df42da2d6119ee14020c665c59d
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyGMs: filename=pyGMs-0.1.1-py3-none-any.whl size=94863 sha256=e90271593def5930eb97a1c782ed5bf0cd28eae15ed94c3537ff6b6d0e6d0313
  Stored in directory: C:\Users\micha\AppData\Local\Temp\pip-ephem-wheel-cache-pdjitg5o\wheels\24\67\25\21d0f2bc6cf79446996c34661dec91090691403e4ac95758ce
Successfully built pyGMs
  Attempting uninstall: pyGMs
    Found existing installation: pyGMs 0.2.0
    Uninstalling pyGMs-0.2.0:
      Successfully uninstalled pyGMs-0.2.0


  Running command git clone --filter=blob:none --quiet https://github.com/ihler/pyGMs.git 'C:\Users\micha\AppData\Local\Temp\pip-req-build-sek7g4cz'


In [2]:
import matplotlib.pyplot as plt

import pyGMs as gm
import numpy as np
import torch

import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist
import pyro.poutine as poutine

import pandas as pd

import requests                                      # reading data
from io import StringIO
import re                                            # regular expressions

# Fix the random seed for reproducibility
# !! Important !! : do not change this
seed = 123
pyro.set_rng_seed(seed)
#np.random.seed(seed);
#torch.manual_seed(seed);

In [3]:
#getting info
df = pd.read_csv('games.csv')


#print(df['white_id'])
white_ids = df['white_id']
black_ids =  df["black_id"]
winners = df["winner"]

print(white_ids[:5])

0         bourgris
1             a-00
2           ischia
3    daniamurashov
4        nik221107
Name: white_id, dtype: object


In [9]:
uniqueppl = white_ids.tolist()
uniqueppl.extend(black_ids.tolist())
#print(len(set(uniqueppl)))




uppl = np.array(list(set(uniqueppl)))#set of unique people

gamesplayed = []
for game in range(0,len(df)):
    p1 = np.argwhere(uppl == df.iloc[game].white_id)[0][0]
    p2 =np.argwhere(uppl == df.iloc[game].black_id)[0][0]

    winner = 0
    if df.iloc[game].winner == "white":
        winner = 1
    if df.iloc[game].winner != "draw":
        gamesplayed.append([p1,p2,winner])

In [10]:
def model(matches):
    
    num_players = max(max(match[0], match[1]) for match in matches)+1
    skill = pyro.sample("X", dist.Normal(torch.zeros(num_players), torch.ones(num_players)))
    for match in matches:
        i, j, outcome = match
        skill_diff = skill[i] - skill[j]
        pyro.sample(f"outcome_{i}_{j}", dist.Bernoulli(logits = skill_diff), obs=torch.tensor(float(outcome)))

In [11]:
matches = gamesplayed[:5]
print(matches)
nuts_kernel = pyro.infer.NUTS(model, jit_compile=False)
mcmc = pyro.infer.MCMC(nuts_kernel,
    num_samples=2000,
    # warmup_steps=100,
    warmup_steps=50,
    num_chains=1)
mcmc.run(matches)

[[10611, 15167, 1], [15167, 4451, 0], [6319, 15167, 1], [1342, 10470, 1], [1205, 10470, 1]]


Sample: 100%|███████████████████████████████████████| 2050/2050 [04:42,  7.27it/s, step size=1.88e-01, acc. prob=0.721]


In [13]:
hmc_samples_chain = {k: v.detach().cpu().numpy() for k, v in mcmc.get_samples().items()}
skillsamp_chain = hmc_samples_chain['X']
x0_gt_x1 = (skillsamp_chain[:, 0] > skillsamp_chain[:, 1])
result = np.sum(x0_gt_x1)/len(x0_gt_x1)
print(f'P(X0>X1): {result}')
x0_gt_x3 = (skillsamp_chain[:, 0] > skillsamp_chain[:, 3])
result = np.sum(x0_gt_x3)/len(x0_gt_x3)
print(f'P(X0>X3): {result}')

P(X0>X1): 0.4875
P(X0>X3): 0.501


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f17de2b6-daef-4780-abae-640fa6a3dc63' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>